# CTiB E2024 - Week 12 - Exercises

# Theoretical exercises

***Exercise 1***: How many terms are there in the sum on slide 13 from the lecture on Nov 18 for computing $P({\bf X}|\Theta)$? Why?

- Z = k^n
where k is the number of states

***Exercise 2***: How many terms are there in the maximization on slide 68 in the Viterbi decoding slides from the lecure on Nov 18 for computing the Viterbi decoding ${\bf Z}^*$? Why?
- k^2 * n

# Practical exercises

You are given the same 7-state HMM and helper functions that you used last week:

In [6]:
class hmm:
    def __init__(self, init_probs, trans_probs, emission_probs):
        self.init_probs = init_probs
        self.trans_probs = trans_probs
        self.emission_probs = emission_probs

In [7]:
init_probs_7_state = [0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00]

trans_probs_7_state = [
    [0.00, 0.00, 0.90, 0.10, 0.00, 0.00, 0.00],
    [1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 0.00, 0.05, 0.90, 0.05, 0.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00],
    [0.00, 0.00, 0.00, 0.10, 0.90, 0.00, 0.00],
]

emission_probs_7_state = [
    #   A     C     G     T
    [0.30, 0.25, 0.25, 0.20],
    [0.20, 0.35, 0.15, 0.30],
    [0.40, 0.15, 0.20, 0.25],
    [0.25, 0.25, 0.25, 0.25],
    [0.20, 0.40, 0.30, 0.10],
    [0.30, 0.20, 0.30, 0.20],
    [0.15, 0.30, 0.20, 0.35],
]

hmm_7_state = hmm(init_probs_7_state, trans_probs_7_state, emission_probs_7_state)

In [8]:
def translate_observations_to_indices(obs):
    mapping = {'a': 0, 'c': 1, 'g': 2, 't': 3}
    return [mapping[symbol.lower()] for symbol in obs]

def translate_indices_to_observations(indices):
    mapping = ['a', 'c', 'g', 't']
    return ''.join(mapping[idx] for idx in indices)

def translate_path_to_indices(path):
    return list(map(lambda x: int(x), path))

def translate_indices_to_path(indices):
    return ''.join([str(i) for i in indices])

# 1 - Viterbi Decoding

Below you will implement and experiment with the Viterbi algorithm. The implementation has been split into three parts:

1. Fill out the $\omega$ table using the recursion presented at the lecture.
2. Find the state with the highest probability after observing the entire sequence of observations.
3. Backtrack from the state found in the previous step to obtain the optimal path.

We'll be working with the 7-state model (`hmm_7_state`) and the helper function for translating between observations, hidden states, and indicies, as introduced above (and also used last week).

Additionally, you're given the function below that constructs a table of a specific size filled with zeros.

In [9]:
def make_table(m, n):
    """Make a table with `m` rows and `n` columns filled with zeros."""
    return [[0] * n for _ in range(m)]

You'll be testing your code with the same two sequences as last week, i.e:

In [10]:
x_short = 'GTTTCCCAGTGTATATCGAGGGATACTACGTGCATAGTAACATCGGCCAA'
z_short = '33333333333321021021021021021021021021021021021021'

In [11]:
x_long = 'TGAGTATCACTTAGGTCTATGTCTAGTCGTCTTTCGTAATGTTTGGTCTTGTCACCAGTTATCCTATGGCGCTCCGAGTCTGGTTCTCGAAATAAGCATCCCCGCCCAAGTCATGCACCCGTTTGTGTTCTTCGCCGACTTGAGCGACTTAATGAGGATGCCACTCGTCACCATCTTGAACATGCCACCAACGAGGTTGCCGCCGTCCATTATAACTACAACCTAGACAATTTTCGCTTTAGGTCCATTCACTAGGCCGAAATCCGCTGGAGTAAGCACAAAGCTCGTATAGGCAAAACCGACTCCATGAGTCTGCCTCCCGACCATTCCCATCAAAATACGCTATCAATACTAAAAAAATGACGGTTCAGCCTCACCCGGATGCTCGAGACAGCACACGGACATGATAGCGAACGTGACCAGTGTAGTGGCCCAGGGGAACCGCCGCGCCATTTTGTTCATGGCCCCGCTGCCGAATATTTCGATCCCAGCTAGAGTAATGACCTGTAGCTTAAACCCACTTTTGGCCCAAACTAGAGCAACAATCGGAATGGCTGAAGTGAATGCCGGCATGCCCTCAGCTCTAAGCGCCTCGATCGCAGTAATGACCGTCTTAACATTAGCTCTCAACGCTATGCAGTGGCTTTGGTGTCGCTTACTACCAGTTCCGAACGTCTCGGGGGTCTTGATGCAGCGCACCACGATGCCAAGCCACGCTGAATCGGGCAGCCAGCAGGATCGTTACAGTCGAGCCCACGGCAATGCGAGCCGTCACGTTGCCGAATATGCACTGCGGGACTACGGACGCAGGGCCGCCAACCATCTGGTTGACGATAGCCAAACACGGTCCAGAGGTGCCCCATCTCGGTTATTTGGATCGTAATTTTTGTGAAGAACACTGCAAACGCAAGTGGCTTTCCAGACTTTACGACTATGTGCCATCATTTAAGGCTACGACCCGGCTTTTAAGACCCCCACCACTAAATAGAGGTACATCTGA'
z_long = '3333321021021021021021021021021021021021021021021021021021021021021021033333333334564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564563210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210321021021021021021021021033334564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564563333333456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456332102102102102102102102102102102102102102102102102102102102102102102102102102102102102102102102103210210210210210210210210210210210210210210210210210210210210210'

Remember to translate these sequences to indices before using them with your algorithms.

## Implementing without log-transformation

First, we will implement the algorithm without log-transformation. This will cause issues with numerical stability (like above when computing the joint probability), so we will use the log-transformation trick to fix this in the next section.

### Computation of the $\omega$ table

In [ ]:
def compute_w(model, x):
    k = len(model.init_probs)
    n = len(x)
    x = translate_observations_to_indices(x)
    w = make_table(k, n)
    #making the first column (the base cases)
    for row in range(0,k): #going over all the k rows
        w[row][0] = model.init_probs[row] * model.emission_probs[row][x[0]] #calculating the base case for each row (initial probability times emission probability)
    #making the rest of the table (the iterative cases)
    for column in range(1, n): #going over all the columns in teh table except teh first one which is already filled out
        for row in range(0, k): #going over all the rows in each column
            w[row][column] = model.emission_probs[row][x[column]] * max((w[h][column-1] * model.trans_probs[h][row]) for h in range(k)) #calculating the value to put in each cell (emission probability times the maximum of the previous state multiplied with the transition probability)   
    return w #return the table


### Finding the joint probability of an optimal path

Now, write a function that given the $\omega$-table, returns the probability of an optimal path through the HMM. As explained in the lecture, this corresponds to finding the highest probability in the last column of the table.

In [29]:
def opt_path_prob(w):
    return max(w[row][-1] for row in range(len(w))) 

Now test your implementation in the box below:

In [30]:
w = compute_w(hmm_7_state, x_short)
opt_path_prob(w)

1.9114255184318858e-31

Now do the same for `x_long`. What happens?

In [31]:
w = compute_w(hmm_7_state, x_long)
opt_path_prob(w)

0.0

### Obtaining an optimal path through backtracking

Implement backtracking to find a most probable path of hidden states given the $\omega$-table.

In [32]:
import math # REMEMBER TO USE math.isclose(a, b) when comparing floats!

In [45]:
def backtrack(model, x, w):
    x = translate_observations_to_indices(x)  # Convert the observations to indices
    path = [] #initialize list
    prob = opt_path_prob_log(w)  # Get the probability
    current_state = [i for i in range(len(w)) if w[i][-1] == prob][0] #accessing the state that matches the prob
    path.insert(0, current_state)  # Add the last state to the path
    k = len(w) #setting the total number of states
    n = len(x) #setting the total number of time steps
    current_time = n - 1  # Start from the last time step
    # Backtrack through the table starting at the next to last column
    for column in range(n - 2, -1, -1):
        for row in range(k):
            # Compute the candidate probability for this row
            prob_candidate = model.emission_probs[current_state][x[current_time]] * w[row][column] * model.trans_probs[row][current_state]
            if math.isclose(prob, prob_candidate):
                previous_state = row #setting the state previous to the current state to be equal to this row
                new_prob = w[row][column] #setting the new_prob to equal what is in this cell in the w table
                if math.isclose(prob_candidate, 0):
                    return "Probability underflow occurred (sequence may be too long or model probabilities too small)"
        path.insert(0, previous_state)  # Insert the previous state into the path
        current_state = previous_state  # Move to the previous state
        current_time = column  # Move to the previous time step
        prob = new_prob #setting prob to equal the value in the w table that matches the new time and state
    return translate_indices_to_path(path)
        

In [46]:
w = compute_w(hmm_7_state, x_short)
z_viterbi = backtrack(hmm_7_state, x_short, w)
z_viterbi

'33333333333321021021021021021021021021021021021021'

Now do the same for `x_long`. What happens?

In [47]:
w = compute_w(hmm_7_state, x_long)
z_viterbi = backtrack(hmm_7_state, x_long, w)
z_viterbi

'Probability underflow occurred (sequence may be too long or model probabilities too small)'

In [48]:
import math

def log(x):
    if x == 0:
        return float('-inf')
    return math.log(x)

## Implementing with log-transformation

Now implement the Viterbi algorithm with log-transformation. The steps are the same as above.

### Computation of the (log-transformed) $\omega$ table

In [49]:
def compute_w_log(model, x):
    x = translate_observations_to_indices(x)
    k = len(model.init_probs)
    n = len(x)
    w = make_table(k, n)
    for row in range(0,k):
        w[row][0] = log(model.init_probs[row]) + log(model.emission_probs[row][x[0]])
    for column in range(1, n):
        for row in range(0, k):
            w[row][column] = log(model.emission_probs[row][x[column]]) + (max((w[h][column-1] + log(model.trans_probs[h][row])) for h in range(k)))
    return w


### Finding the (log-transformed) joint probability of an optimal path

In [50]:
def opt_path_prob_log(w):
    k = len(w)
    max_val = float('-inf')
    for i in range(k):
        if w[i][-1] > max_val:
            max_val = w[i][-1]
    return max_val  

In [51]:
w = compute_w_log(hmm_7_state, x_short)
opt_path_prob_log(w)

-70.73228857440488

Now do the same for `x_long`. What happens?

In [52]:
w = compute_w_log(hmm_7_state, x_long)
opt_path_prob_log(w)

-1406.7209253880144

### Obtaining an optimal path through backtracking

In [53]:
import math

def backtrack_log(model, x, w):
    x = translate_observations_to_indices(x)  # Convert the observations to indices
    path = [] #initialize list
    prob = opt_path_prob_log(w)  # Get the probability
    current_state = [i for i in range(len(w)) if w[i][-1] == prob][0] #accessing the state that matches the prob
    path.insert(0, current_state)  # Add the last state to the path
    k = len(w) #setting the total number of states
    n = len(x) #setting the total number of time steps
    current_time = n - 1  # Start from the last time step
    # Backtrack through the table starting at the next to last column
    for column in range(n - 2, -1, -1):
        for row in range(k):
            # Compute the candidate probability for this row
            prob_candidate = log(model.emission_probs[current_state][x[current_time]]) + w[row][column] + log(model.trans_probs[row][current_state])
            if math.isclose(prob, prob_candidate):
                previous_state = row #setting the state previous to the current state to be equal to this row
                new_prob = w[row][column] #setting the new_prob to equal what is in this cell in the w table
                if math.isclose(prob_candidate, -math.inf):
                    return "Probability underflow occurred (sequence may be too long or model probabilities too small)"
        path.insert(0, previous_state)  # Insert the previous state into the path
        current_state = previous_state  # Move to the previous state
        current_time = column  # Move to the previous time step
        prob = new_prob #setting prob to equal the value in the w table that matches the new time and state
    return translate_indices_to_path(path)


In [54]:
w = compute_w_log(hmm_7_state, x_short)
z_viterbi_log = backtrack_log(hmm_7_state, x_short, w)
z_viterbi_log

'33333333333321021021021021021021021021021021021021'

Now do the same for `x_long`. What happens?

In [44]:
w = compute_w_log(hmm_7_state, x_long)
z_viterbi_log = backtrack_log(hmm_7_state, x_long, w)
z_viterbi_log

'333332102102102102102102102102102102102102102102102102102102102102102103333333333456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456321021021021021021021021021021021021021021021021021021021021021021021021021021021021021021021021021021021021021021021021021021021032102102102102102102102103333456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456333333345645645645645645645645645645645645645645645645645645645645645645645645645645645645645645645645645645645645645645633210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210321021021021021021021021021021021021021021021021021021021021021

### Does it work?

Think about how to verify that your implementations of Viterbi (i.e. `compute_w`, `opt_path_prob`, `backtrack`, and there log-transformed variants `compute_w_log`, `opt_path_prob_log`, `backtrack_log`) are correct.

One thing that should hold is that the probability of a most likely path as computed by `opt_path_prob` (or `opt_path_prob_log`) for a given sequence of observables (e.g. `x_short` or `x_long`) should be equal to the joint probability of a corersponding most probable path as found by `backtrack` (or `backtrack_log`) and the given sequence of observables. Why?

Make an experiment that validates that this is the case for your implementations of Viterbi and `x_short` and `x_long`.

In [62]:
# To access joint_prob and joint_prob_log, you must copy your implementations from last week here ...

def joint_prob(model, x, z):
    prob = model.init_probs[z[0]]
    for i in range(1, len(z)):
        prob *= model.trans_probs[z[i-1]][z[i]]
    for i in range(len(z)):
        prob *= model.emission_probs[z[i]][x[i]]  
    return prob

def joint_prob_log(model, x, z):
    prob = log(model.init_probs[z[0]])
    for i in range(1, len(z)):
        prob += log(model.trans_probs[z[i-1]][z[i]])
    for i in range(len(z)):
        prob += log(model.emission_probs[z[i]][x[i]])  
    return prob


# Check that opt_path_prob is equal to joint_prob(hmm_7_state, x_short, z_viterbi)
print(math.isclose(opt_path_prob(compute_w(hmm_7_state, x_short)), joint_prob(hmm_7_state, translate_observations_to_indices(x_short), translate_path_to_indices(z_short))))

# Check that opt_path_prob_log is equal to joint_prob_log(hmm_7_state, x_short, z_viterbi_log)
print(math.isclose(opt_path_prob_log(compute_w_log(hmm_7_state, x_short)), joint_prob_log(hmm_7_state, translate_observations_to_indices(x_short), translate_path_to_indices(z_short))))

# Do the above checks for x_long ...
print(math.isclose(opt_path_prob(compute_w(hmm_7_state, x_long)), joint_prob(hmm_7_state, translate_observations_to_indices(x_long), translate_path_to_indices(z_long))))
print(math.isclose(opt_path_prob_log(compute_w_log(hmm_7_state, x_long)), joint_prob_log(hmm_7_state, translate_observations_to_indices(x_long), translate_path_to_indices(z_long))))

True
True
True
True


Do your implementations pass the above checks?

Yes, but only because the third print statement ends up comparing 0 and 0. So even though it passes, it compares two wrong values (which both happen to be 0, since the probabilities have become so small)...

### Does log-transformation matter?

Make an experiment that investigates how long the input string can be before `backtrack` and `backtrack_log` start to disagree on a most likely path and its probability.

In [66]:
for i in range(1, len(x_long), 10):
    x = x_long[:i]
    z = z_long[:i]
    
    x_trans = translate_observations_to_indices(x)
    z_trans = translate_path_to_indices(z)
    w = compute_w(hmm_7_state, x)
    w_log = compute_w_log(hmm_7_state, x)

    no_log = backtrack(hmm_7_state, x, w)
    with_log = backtrack_log(hmm_7_state, x, w_log)
    if not no_log == with_log:
        print(i)
        break

531


**Your answer here:**

For the 7-state model, `backtrack` and `backtrack_log` start to disagree on a most likely path and its probability for **i = 524** if you run it with increments of 1. If you run it with increments of 10 (which is way faster and what we have done above) you get an approximate value of **i = 531**. 